In [1]:
%pwd

'D:\\ML\\NewYorkTaxi_Jupyter'

In [2]:
import pandas as pd

training = pd.read_csv('data/train.csv', nrows=100)
test = pd.read_csv('data/test.csv', nrows=100)

pd.set_option('display.max_columns',10)
pd.set_option('display.width', 100)

print(training.pickup_datetime.head())
type(training)

training.isnull().values.any()


0    2009-06-15 17:26:21 UTC
1    2010-01-05 16:52:16 UTC
2    2011-08-18 00:35:00 UTC
3    2012-04-21 04:30:42 UTC
4    2010-03-09 07:51:00 UTC
Name: pickup_datetime, dtype: object


False

We need to write a custom function for cross validation.

Ideas:
0- Check for empty and null values
1- Simplify pick time to a number between 0 to 24 to just represent time of the day
2- Come up with a new column that shows the distance. This may not be great since the exact locations may affect the length of the trip that could affect the fare.
3- Check for linear correlation

In [1]:
training = pd.read_csv('data/train.csv', on_bad_lines='skip', nrows=5000) # TODO remove limit
test = pd.read_csv('data/test.csv')

print("Data loaded.")
print("Read " + str(len(training)) + " training rows")
print("Read " + str(len(test)) + " test rows")

NameError: name 'pd' is not defined

Now we break apart the pickup times.

In [56]:
def convert_pickup_times(df):
    df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'], format='%Y-%m-%d %H:%M:%S UTC', errors='coerce')

def break_pickup_times_to_components(df):
    df['pickup_year'] = df['pickup_datetime'].dt.year
    df['pickup_month'] = df['pickup_datetime'].dt.month
    df['pickup_day'] = df['pickup_datetime'].dt.day
    df['pickup_hour'] = df['pickup_datetime'].dt.hour
    df['pickup_minute'] = df['pickup_datetime'].dt.minute
    #converting seconds sounds like an overkill so I won't use it
    
    #but day of week sounds like a predictive feature
    df['pickup_dayofweek'] = df['pickup_datetime'].dt.dayofweek
    
    #remove the redundant col
    df.drop(columns=['pickup_datetime'], inplace=True)

def prepare(df):
    convert_pickup_times(df)
    break_pickup_times_to_components(df)
    df.drop(columns=['key'], inplace=True)
    
prepare(training)
prepare(test)

pd.set_option('display.max_columns',16)
pd.set_option('display.width', 100)
print(training.head())
print(test.head())

   fare_amount  pickup_longitude  pickup_latitude  dropoff_longitude  dropoff_latitude  \
0          4.5        -73.844311        40.721319         -73.841610         40.712278   
1         16.9        -74.016048        40.711303         -73.979268         40.782004   
2          5.7        -73.982738        40.761270         -73.991242         40.750562   
3          7.7        -73.987130        40.733143         -73.991567         40.758092   
4          5.3        -73.968095        40.768008         -73.956655         40.783762   

   passenger_count  pickup_year  pickup_month  pickup_day  pickup_hour  pickup_minute  \
0                1         2009             6          15           17             26   
1                1         2010             1           5           16             52   
2                2         2011             8          18            0             35   
3                1         2012             4          21            4             30   
4             

5
0


Since the number of cells with null value is relatively small and there's no null values in the test data we can just drop them.

In [59]:
def check_for_nulls(df):
    is_null = df.isnull()
    row_has_null = is_null.any(axis=1)
    print(row_has_null.sum())
    
check_for_nulls(training)
check_for_nulls(test)

training = training[~training.isnull().any(axis=1)]
print(str(len(training)) + " rows")

0
0
499995 rows


In [ ]:
from sklearn import svm
from sklearn import model_selection
import numpy as np

def custom_cross_validation(cv, x_train, y_train, model):
    pie_len = len(x_train) / cv
    total_len = len(x_train)
    for i in range(0, cv):
        b = int(i * pie_len)
        e = int(b + pie_len)
        train_range = np.r_[0:b, e:total_len]
        test_range = np.r_[b:e]
        model.fit(x_train[train_range], y_train[train_range])
        predictions = model.predict_proba(x_train[test_range])

        rmse = mean_squared_error(y_train[test_range], predictions, squared=False)

        print(rmse)

X = training.iloc[:, training.columns != 'fare_amount'].values
y = training.iloc[:, training.columns == 'fare_amount'].values.ravel()

print("Data was split into features and labels.")

regr = svm.SVR()

custom_cross_validation(5, X, y, regr)



Data was split into features and labels.


In [ ]:
regr.fit(X, y)